In [135]:
from rosemary import jpt_setup; jpt_setup()


import json
import random
import re
import os
import pandas as pd
import argparse
from open_instruct.instruction_encode_templates import encode_instruction_example, encode_few_shot_example
from open_instruct.reformat_datasets import get_all_supported_datasets

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:



"""

python open_instruct/reformat_datasets.py \
    --raw_data_dir data/raw_train/ \
    --output_dir data/processed/ \
    --dataset flan_v2_SirNeural

"""


raw_data_dir = 'data/raw_train/'
output_dir = 'data/processed/'
seed = 42






In [132]:

# get the subfolder names in raw_data_dir
subfolders = [f for f in os.listdir(raw_data_dir) if os.path.isdir(os.path.join(raw_data_dir, f))]


subfolders


['cot',
 'sharegpt',
 'open_orca',
 'dolly',
 'gpt4_alpaca',
 'flan_v2',
 'wizardlm',
 'super_ni',
 'lima',
 'stanford_alpaca',
 'baize',
 'code_alpaca',
 'flan_v2_SirNeural',
 'self_instruct',
 'unnatural_instructions',
 'oasst1']

In [133]:
supported_datasets = get_all_supported_datasets()
        
supported_datasets

['super_ni',
 'cot',
 'flan_v2',
 'flan_v2_SirNeural',
 'dolly',
 'self_instruct',
 'unnatural_instructions',
 'stanford_alpaca',
 'code_alpaca',
 'gpt4_alpaca',
 'sharegpt',
 'baize',
 'oasst1',
 'lima',
 'wizardlm',
 'open_orca']

In [134]:

# check if the subfolder names are supported datasets
valid_subfolders = []
for subfolder in subfolders:
    if subfolder not in supported_datasets:
        print(f"Warning: {subfolder} in the raw data folder is not a supported dataset. We will skip it.")
    else:
        valid_subfolders.append(subfolder)
valid_subfolders

['cot',
 'sharegpt',
 'open_orca',
 'dolly',
 'gpt4_alpaca',
 'flan_v2',
 'wizardlm',
 'super_ni',
 'lima',
 'stanford_alpaca',
 'baize',
 'code_alpaca',
 'flan_v2_SirNeural',
 'self_instruct',
 'unnatural_instructions',
 'oasst1']

In [39]:
# def convert_flan_v2_data(data_dir, output_dir):
subfolder = 'flan_v2'
data_dir = os.path.join(raw_data_dir, subfolder)
output_dir = os.path.join(output_dir, subfolder)

print(data_dir)
print(output_dir)


os.makedirs(output_dir, exist_ok=True)
examples = []


with open(os.path.join(data_dir, "flan_v2_resampled_100k.jsonl"), "r") as fin:
    for line in fin:
        examples.append(json.loads(line))
        
        

len(examples), set([x['_template_type'] for x in examples])

data/raw_train/flan_v2
data/processed/flan_v2_SirNeural/flan_v2_SirNeural/flan_v2_SirNeural/flan_v2_SirNeural/flan_v2_SirNeural/flan_v2/flan_v2/flan_v2/flan_v2


(100000, {'fs_noopt', 'fs_opt', 'zs_noopt', 'zs_opt'})

In [32]:
for idx, example in enumerate(examples):
    prompt = example["inputs"]
    if not prompt.endswith("\n") and not prompt.rstrip().endswith(":"):
        prompt += "\n"
    completion = example["targets"]
    sample = {
        "dataset": "flan_v2",
        "id": f"flan_v2_{idx}",
        "messages": [
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": completion},
        ]
    }
    break
#     fout.write(json.dumps({
#         "dataset": "flan_v2",
#         "id": f"flan_v2_{idx}",
#         "messages": [
#             {"role": "user", "content": prompt},
#             {"role": "assistant", "content": completion},
#         ]
#     }) + "\n")
    

### flan_v2_SirNeural


In [136]:

subfolder = 'flan_v2_SirNeural'
data_dir = os.path.join('data/raw_train/', subfolder)
output_dir = os.path.join('data/processed/', subfolder)

print(data_dir, '\n', output_dir)


#### 

input_files = [x for x in os.listdir(data_dir) if x.endswith('jsonl.gz')]

for input_file in input_files:
    input_path = os.path.join(data_dir, input_file)
    output_path = os.path.join(output_dir, input_file.split('_train.jsonl.gz')[0]+'_data.jsonl')

    if os.path.isfile(output_path):
        continue
        
    print(f'Processing {input_file}')

    ds = load_dataset('json', data_files={'train': input_path}, split='train')
    # ds = ds.select(range(100))

    def convert_data_fn(example, idx):
        prompt = example['inputs']
        if not prompt.endswith("\n") and not prompt.rstrip().endswith(":"):
            prompt += "\n"
        completion = example["targets"]
        return {
            'dataset': "flan_v2", 
            'id': f"flan_v2_{idx}",
            'messages': [
                {'role': 'user', 'content': prompt},
                {"role": "assistant", "content": completion},
            ]}
    ds = ds.map(convert_data_fn, 
                remove_columns=["inputs", "targets", "task"], 
                with_indices=True,
                num_proc=30,
                desc=f'Convert data for {input_file}',
                keep_in_memory=True)

    ds.to_json(output_path)


data/raw_train/flan_v2_SirNeural 
 data/processed/flan_v2_SirNeural
Processing flan_fs_opt_train_part1.jsonl.gz


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Failed to read file '/gpfs/u/scratch/PTFM/PTFMqngp/huggingface_cache/datasets/downloads/extracted/1d4f82a7be082808c68105ea6d1c118c2eb8c6b6e629634d0cbd880ccbdbe6fd' with error <class 'pyarrow.lib.ArrowInvalid'>: JSON parse error: Missing a closing quotation mark in string. in row 87


DatasetGenerationError: An error occurred while generating the dataset

In [143]:
examples 
n = 0
with gzip.open('data/raw_train/flan_v2_SirNeural/flan_fs_opt_train_part1.jsonl.gz', 'rt') as f:
    for line in f:
        if n%100000 == 0:
            print(n)
#         json.loads(line)
#         examples.append(json.loads(line))
        n += 1

0
100000
200000



KeyboardInterrupt



In [ ]:
50 048 610

In [122]:

input_file = 'cot_fs_noopt_train.jsonl.gz'
input_path = os.path.join(data_dir, input_file)
output_path = os.path.join(output_dir, input_file.split('_train.jsonl.gz')[0]+'_data.jsonl')

ds = load_dataset('json', data_files={'train': input_path}, split='train', streaming=True)

In [125]:


def convert_data_fn(example, idx):
    prompt = example['inputs']
    if not prompt.endswith("\n") and not prompt.rstrip().endswith(":"):
        prompt += "\n"
    completion = example["targets"]
    return {
        'dataset': "flan_v2", 
        'id': f"flan_v2_{idx}",
        'messages': [
            {'role': 'user', 'content': prompt},
            {"role": "assistant", "content": completion},
        ]}


ds = ds.map(convert_data_fn, 
            remove_columns=["inputs", "targets", "task"], 
            with_indices=True,
            num_proc=30)


TypeError: IterableDataset.map() got an unexpected keyword argument 'num_proc'

In [117]:
python scripts/resample_flan_v2.py --flan_v2_data_dir data/raw_train/flan_v2_SirNeural --total_num_samples 1000000 --output_path data/processed/flan_v2_SirNeural/flan_v2_1m.jsonl


100